In [1]:
import sys
from driftdb.connectors.github_connector import GithubConnector
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../../.env")

gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
github_client = Github(gh_token, timeout=60)
repo_name = os.getenv("REPO") or "gh_org/repo"
github_connector = GithubConnector(github_client=github_client, github_repository_name=repo_name, assignees=["Sammy"])


driftdb.connectors.github_connector - WARNING - Assignee Sammy does not exist


In [2]:
import pandas as pd
from faker import Faker
import random

fake = Faker()

num_rows = 60000

ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1y', end_date='-1m').strftime('%Y-%m-%d') for _ in range(num_rows)]

metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
country_codes = [fake.country_code() for _ in range(num_rows)]
categories = [random.choice(['Category A', 'Category B', 'Category C']) for _ in range(num_rows)]

ultra_large_df = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values, 'country_code': country_codes, 'category': categories})

print(ultra_large_df)



                                 unique_key        date  metric_value  \
0      08b6cd5b-13aa-4fbd-ac34-d87ed2787add  2023-01-03          0.23   
1      1fb22c21-59d7-4f6e-ba47-a7e54a4d2e5e  2023-03-19          0.41   
2      209d6e40-a8f7-47ed-8832-70c495add816  2023-04-04          0.31   
3      a9273006-184f-494c-8c7d-eb7332c82748  2023-03-20          9.01   
4      6d804ede-a68e-4a06-a6ab-2f92343b96b5  2023-05-07          6.19   
...                                     ...         ...           ...   
59995  5d44f1dd-2ae2-4d52-ab8c-51ef8d03b320  2023-07-07          1.65   
59996  ec22ba13-808a-4aa3-8ef1-2fab63789463  2022-12-14          8.88   
59997  558d74f0-600d-4c56-b7ca-622ea22f2444  2023-06-07          0.31   
59998  0dab2068-79e3-41e6-ae04-8b19691c7a8a  2023-01-26          9.75   
59999  9fc73095-e90c-4d4a-8da8-5f9c528ad40b  2023-01-20          6.42   

      country_code    category  
0               NE  Category A  
1               VA  Category C  
2               AU  Cate

In [3]:
import sys

## Test with file already existing and splitting new data and historical data
table_name = "path/to/weekly_partition_2"
github_connector.partition_and_snapshot_table(
    table_dataframe= ultra_large_df,
    table_name= table_name,
    freq="W",
)

driftdb.connectors.github_connector - INFO - Partitionning table by month...
driftdb.connectors.github_connector - INFO - Storing table for: 2022-12-18 00:00:00
/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/driftdb/driftdb/dataframe/helpers.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unique_key"] = df["unique_key"].astype(str)
/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/driftdb/driftdb/dataframe/helpers.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

ultra_large_df2 = ultra_large_df.copy()

# Select 10 random indices for metric value update
random_indices_metric = np.random.choice(ultra_large_df2.index, size=100, replace=False)
print(random_indices_metric)
# Update metric value with random values between 0 and 10
ultra_large_df2.loc[random_indices_metric, 'metric_value'] = [round(random.uniform(0, 10),2) for _ in range(100)]


[12628 37730 39991  8525  8279 51012 14871 15127  9366 33322 53390 21819
  5026 23428 45297 26354 30195 47038 20731 34047 26064 42469 29746 14522
 31572 54949 19368  3803 53325 14300 51301  9008 47521 25224 48921 37978
 44171 26303 19458  5369 50291 25951 54908 56362 32218  2885 36559  8966
 46574 10530 44628   273 19269 36911 10121 13290 57606 47189 29209 42187
 25386 17005 10981 47313 27070  6685 54960 58125 40700 13902 31539 49716
 49519 51923  3502 39336  2218 18505 10689 21377  1866 20192 28870 52203
  3867  3222 21785 20984 48539 40694  8440  9951  1334 32572 28344 46503
 34482 36271 36874 11512]


In [ ]:
ultra_large_df.dtypes

unique_key              object
date            datetime64[ns]
metric_value           float64
country_code            object
category                object
dtype: object

In [ ]:
import sys
sys.path.append('..')
import importlib

import driftdb.drift_evaluators
importlib.reload(driftdb.drift_evaluators)
from driftdb.drift_evaluators import auto_merge_drift


import driftdb.github_connector
importlib.reload(driftdb.github_connector)
from driftdb.github_connector import partition_and_store_table


partition_and_store_table(gh_client,  ultra_large_df2, table_name)

ModuleNotFoundError: No module named 'driftdb.drift_evaluators'

In [ ]:
# Set the number of rows for the dataframe
num_rows = 6000

# Generate random IDs and dates
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1m', end_date='today').strftime('%Y-%m-%d') for _ in range(num_rows)]

# Generate random metric values between 0 and 10
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
# Generate random country codes
country_codes = [fake.country_code() for _ in range(num_rows)]

# Generate random categories
categories = [random.choice(['Category A', 'Category B', 'Category C']) for _ in range(num_rows)]

# Create the dataframe
new_lines = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values, 'country_code': country_codes, 'category': categories})

ultra_large_df3 = pd.concat([ultra_large_df2, new_lines], ignore_index=True)

In [ ]:
import sys
sys.path.append('..')
import importlib

import driftdb.drift_evaluators
importlib.reload(driftdb.drift_evaluators)
from driftdb.drift_evaluators import default_drift_evaluator


import driftdb.github_connector
importlib.reload(driftdb.github_connector)
from driftdb.github_connector import store_metric

store_metric(ghClient=gh_client,  dataframe=ultra_large_df3, filepath=table_name, assignees=["Sammy"], store_json=False)

Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name18.csv?token=ABUWFP5KQIF72QPM6CKZ6VTEW2IZK
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
New data found
Commit: New data: path/to/ultra_large_metric_name18.csv
https://github.com/Samox/data-history/commit/510cc1a3978ec68cc439c5f19088d9626eb0db2f
Branch metric/path-to-ultra-large-metric-name18-csv doesn't exist, creating it...
comparison                                      metric_value      
                                             self other
unique_key                                             
03aeb072-4c9b-4900-a65f-0cb012459f50         5.28 

In [ ]:
ultra_large_df.to_json(local_file_path+'.json')
